In [ ]:
import os

def scale_input_data(scale_factor):
  file_bases = ['./input/melb_data']
  for file_base in file_bases:
    import pandas as pd
    import shutil
    if scale_factor == 1.0:
      shutil.copyfile(file_base + '.csv', file_base + '.scaled.csv')
      continue
    df_to_scale = pd.read_csv(file_base + '.csv')
    new_num_rows = int(scale_factor * len(df_to_scale))
    if scale_factor <= 1.0:
      df_to_scale = df_to_scale.iloc[:new_num_rows]
    else:
      while len(df_to_scale) < new_num_rows:
        df_to_scale = pd.concat([df_to_scale, df_to_scale[:min(new_num_rows - len(df_to_scale), len(df_to_scale))]])
    df_to_scale.to_csv(file_base + '.scaled.csv', index=False)

if 'INPUT_SCALE_FACTOR' in os.environ:
  scale_input_data(float(os.environ['INPUT_SCALE_FACTOR']))

Notebook below presents Exploratory data analysis on Melbourne house pricing dataset. 

## Data Provided

Rooms: Number of rooms

Price: Price in dollars

Method: S - property sold; SP - property sold prior; PI - property passed in; PN - sold prior not disclosed; SN - sold not disclosed; NB - no bid; VB - vendor bid; W - withdrawn prior to auction; SA - sold after auction; SS - sold after auction price not disclosed. N/A - price or highest bid not available.

Type: br - bedroom(s); h - house,cottage,villa, semi,terrace; u - unit, duplex; t - townhouse; dev site - development site; o res - other residential.

SellerG: Real Estate Agent

Date: Date sold

Distance: Distance from CBD

Regionname: General Region (West, North West, North, North east …etc)

Propertycount: Number of properties that exist in the suburb.

Bedroom2 : Scraped # of Bedrooms (from different source)

Bathroom: Number of Bathrooms

Car: Number of carspots

Landsize: Land Size

BuildingArea: Building Size

CouncilArea: Governing council for the area

In [1]:
#importing libraries
# import pandas as pd
exec(os.environ['IREWR_IMPORTS'])
import numpy as np
# ALEX: remove plotting
# import seaborn as sns
# import matplotlib
# from matplotlib import pyplot as plt
# %matplotlib inline
pd.pandas.set_option('display.max_columns',None)
pd.pandas.set_option('display.max_rows',None)
import warnings
warnings.filterwarnings('ignore')

In [2]:
#loading dataset
data=pd.read_csv('./input/melb_data.scaled.csv')
df=data.copy()

In [3]:
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


### Descriptive Statistics 

In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Rooms,13580.0,2.937997e+00,0.955748,1.00000,2.000000,3.000000,3.000000e+00,1.000000e+01
Price,13580.0,1.075684e+06,639310.724296,85000.00000,650000.000000,903000.000000,1.330000e+06,9.000000e+06
Distance,13580.0,1.013778e+01,5.868725,0.00000,6.100000,9.200000,1.300000e+01,4.810000e+01
Postcode,13580.0,3.105302e+03,90.676964,3000.00000,3044.000000,3084.000000,3.148000e+03,3.977000e+03
Bedroom2,13580.0,2.914728e+00,0.965921,0.00000,2.000000,3.000000,3.000000e+00,2.000000e+01
Bathroom,13580.0,1.534242e+00,0.691712,0.00000,1.000000,1.000000,2.000000e+00,8.000000e+00
Car,13518.0,1.610075e+00,0.962634,0.00000,1.000000,2.000000,2.000000e+00,1.000000e+01
Landsize,13580.0,5.584161e+02,3990.669241,0.00000,177.000000,440.000000,6.510000e+02,4.330140e+05
BuildingArea,7130.0,1.519676e+02,541.014538,0.00000,93.000000,126.000000,1.740000e+02,4.451500e+04
YearBuilt,8205.0,1.964684e+03,37.273762,1196.00000,1940.000000,1970.000000,1.999000e+03,2.018000e+03


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         13580 non-null  object 
 1   Address        13580 non-null  object 
 2   Rooms          13580 non-null  int64  
 3   Type           13580 non-null  object 
 4   Price          13580 non-null  float64
 5   Method         13580 non-null  object 
 6   SellerG        13580 non-null  object 
 7   Date           13580 non-null  object 
 8   Distance       13580 non-null  float64
 9   Postcode       13580 non-null  float64
 10  Bedroom2       13580 non-null  float64
 11  Bathroom       13580 non-null  float64
 12  Car            13518 non-null  float64
 13  Landsize       13580 non-null  float64
 14  BuildingArea   7130 non-null   float64
 15  YearBuilt      8205 non-null   float64
 16  CouncilArea    12211 non-null  object 
 17  Lattitude      13580 non-null  float64
 18  Longti

#### Missing values 

In [6]:
# ALEX: remove plotting
# plt.figure(figsize=(10,8))
# sns.heatmap(df.isnull(), cbar=False,cmap="Greens");
_ = df.isnull()

We can see a large number of missing values in Building area and year built. For now I am dropping column with building area as almost half of the data is missing. Also I am dropping address column as it is same as having Id column. 

In [7]:
# ALEX: make notebook run with input scaling
# df.drop(['BuildingArea','Address'],axis=1).head(5)
df.drop(['BuildingArea','Address'],axis=1,errors='ignore').head(5)

,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


Separating features based on their data types for further analysis

In [8]:
numerical_feature=[feature for feature in df.columns if df[feature].dtype !='object']
print('There are {} numerical features.'.format(len(numerical_feature)))

year=[feature for feature in numerical_feature if "Yr" in feature or "Year" in feature]

discrete_feature=[feature for feature in numerical_feature if (df[feature].nunique())<25 and feature not in year+['Id']]
print('There are {} are discrete features'.format(len(discrete_feature)) )

continuous_feature=[feature for feature in numerical_feature if feature not in discrete_feature and feature not in year+['Id']]
print('There are {} are continuous features'.format(len(continuous_feature)) )

cat_feature=[feature for feature in df.columns if df[feature].dtypes=='object']
print('There are {} categorical features.'.format(len(cat_feature)))

There are 13 numerical features.
There are 4 are discrete features
There are 8 are continuous features
There are 8 categorical features.


Replacing names in columns

In [9]:
type={'h':'house','u':'unit','t':'townhouse'}
df=df.replace({'Type':type})

### Numerical features

Outliers in continuous numeric features:

In [10]:
def outlier_function(df, col_name):
    first_quartile = np.percentile(np.array(df[col_name].tolist()), 25)
    third_quartile = np.percentile(np.array(df[col_name].tolist()), 75)
    IQR = third_quartile - first_quartile
                      
    upper_limit = third_quartile+(3*IQR)
    lower_limit = first_quartile-(3*IQR)
    outlier_count = 0
                      
    for value in df[col_name].tolist():
        if (value < lower_limit) | (value > upper_limit):
            outlier_count +=1
    return lower_limit, upper_limit, outlier_count
for feature in continuous_feature:
    if outlier_function(df,feature)[2] > 1:
        print('{}: {} outliers'.format(feature,outlier_function(df,feature)[2]))

Price: 132 outliers
Distance: 130 outliers
Postcode: 111 outliers
Landsize: 221 outliers
Lattitude: 27 outliers
Longtitude: 8 outliers


Let us now take a look at correlations within continuous features with the help of a heatmap. Later we shall see each of the feature in detail.

In [11]:
# ALEX: remove plotting
# plt.figure(figsize=(10,8))
corr=df[continuous_feature].corr()
# ALEX: remove plotting
# plt.title('Correlation heatmap of continuous features',fontsize=15)
# sns.heatmap(corr);

The above heatmap shows correlation between the above factors . 

There is a negative correlation between price and distance from CBD, it means that as people move farther away from central business district the house prices decrease. 


#### Distribution of observations

In [12]:
# ALEX: remove plotting
# fig, axarr = plt.subplots(2, 2, figsize=(12, 10))

# sns.distplot(df['Price'],ax=axarr[0][0],color='blue',hist_kws=dict(edgecolor="k", linewidth=2)).set_title('Distribution of Price')
# sns.distplot(df['Distance'],ax=axarr[1][0],color='green',hist_kws=dict(edgecolor="k", linewidth=2)).set_title('Distribution of Distance')
# sns.distplot(np.log(df['Price']),ax=axarr[0][1],color='red',hist_kws=dict(edgecolor="k", linewidth=2)).set_title('Distribution of Price(Transformed value on log scale)')
# sns.distplot(df['Landsize'],ax=axarr[1][1],color='orange',hist_kws=dict(edgecolor="k", linewidth=2)).set_title('Distribution of Landsize')
# plt.subplots_adjust(hspace=.6)
# sns.set_style('darkgrid')
# sns.despine()

_ = df['Price']
_ = df['Distance']
_ = np.log(df['Price'])
_ = df['Landsize']

We can see that log tranformed values of price has considerably reduced skew. thus for few plots we shall make use of log transforme dvalues of price.

In [13]:
df['price_per_unit_area']=df['Price']/df['Landsize']

In [14]:
# ALEX: remove plotting
# plt.figure(figsize=(10,8))
# sns.scatterplot(y=np.log(df['price_per_unit_area']),x=df['Distance'],data=df,hue='Regionname').set_title("Distance from property to CBD",fontsize=20);
_ = np.log(df['price_per_unit_area'])
_ = df['Distance']

Price of the houses on an average are same at distance farther away from CBD. But the number of houses sold are less as we move away from CBD. Also  Northern Metropolitan, Southern Metropolitan and Western Metropolitan regions are nearer to CBD

### Discrete features 

In [15]:
df[discrete_feature].head(3)

,Rooms,Bedroom2,Bathroom,Car
0,2,2.0,1.0,1.0
1,2,2.0,1.0,0.0
2,3,3.0,2.0,0.0


In [16]:
for feature in discrete_feature:
    if outlier_function(df,feature)[2] > 1:
        print('{}: {} outliers'.format(feature,outlier_function(df,feature)[2]))

Rooms: 19 outliers
Bedroom2: 20 outliers
Bathroom: 9 outliers


In [17]:
df['Bedroom2']=df['Bedroom2'].astype('int')
df['Bathroom']=df['Bathroom'].astype('int')
df['Car']=df['Car'].astype('int',errors='ignore')
df['Car']=df['Car'].astype('int',errors='ignore')

In [18]:
# ALEX: remove plotting
# fig, axarr = plt.subplots(2, 2, figsize=(14,10))
# fig.suptitle('Property features',fontsize=15)

# sns.countplot(df['Rooms'],ax=axarr[0][0],palette='cubehelix').set_title('No. of Rooms')
# sns.countplot(df['Bedroom2'],ax=axarr[1][0],palette='cubehelix').set_title('No. of Bedrooms')
# sns.boxplot(x=df['Bedroom2'],y=df['Price'],ax=axarr[1][1],palette='husl').set_title('Price with respect to number of bedrooms')
# sns.boxplot(x=df['Rooms'],y=df['Price'],ax=axarr[0][1],palette='husl').set_title('Price with respect to no. of rooms')

# plt.subplots_adjust(hspace=.4)
# sns.set_style('darkgrid')
# sns.despine()
_ = df['Rooms']
_ = df['Bedroom2']
_ = df['Bedroom2']
_ = df['Price']
_ = df['Rooms']
_ = df['Price']

There is no much variation in context of rooms and bedrooms. Houses that are most preffered have 2-3 rooms. Pricier houses have room count of 5.

In [19]:
# ALEX: remove plotting
# fig, axarr = plt.subplots(2, 2, figsize=(14, 10))
# fig.suptitle('Property features',fontsize=15)

# sns.countplot(df['Bathroom'],ax=axarr[0][0],palette='husl').set_title('No. of Bathrooms')
# sns.violinplot(x= "Bathroom",y="Price",data=df,palette="Set2",ax=axarr[0][1]).set_title('No. of Bathrooms vs Price')
# sns.countplot(df['Car'],ax=axarr[1][0],palette='husl').set_title('No.of Cars')
# sns.boxenplot(x="Car",y="Price",data=df,palette="Set1",ax=axarr[1][1]).set_title('No.of Cars vs Price');


# plt.subplots_adjust(hspace=.4)
# sns.set_style('darkgrid')
# sns.despine()

_ = df['Bathroom']
_ = df['Car']

Properties, most commonly have either 1 or 2 bathrooms. Houses with more than 4 bathrooms tend to be expensive.Same trend goes iwth the number of cars.

### Categorical features 

In [20]:
df[cat_feature].head()

,Suburb,Address,Type,Method,SellerG,Date,CouncilArea,Regionname
0,Abbotsford,85 Turner St,house,S,Biggin,3/12/2016,Yarra,Northern Metropolitan
1,Abbotsford,25 Bloomburg St,house,S,Biggin,4/02/2016,Yarra,Northern Metropolitan
2,Abbotsford,5 Charles St,house,SP,Biggin,4/03/2017,Yarra,Northern Metropolitan
3,Abbotsford,40 Federation La,house,PI,Biggin,4/03/2017,Yarra,Northern Metropolitan
4,Abbotsford,55a Park St,house,VB,Nelson,4/06/2016,Yarra,Northern Metropolitan


In [21]:
# ALEX: remove plotting
# plt.figure(figsize=(12,8))
# g=sns.countplot(df['CouncilArea'],data=df,palette='Set2')
# g.set_xticklabels(g.get_xticklabels(), rotation=90)
# g.set_title('No.of properties in each Council Area',fontsize=15);
_ = df['CouncilArea']

* Council Area with most properties sold was Moreland and Boroondara , while council Moorabool saw the least sales. 

In [22]:
# ALEX: remove plotting
# plt.figure(figsize=(10,8))
# g=sns.stripplot(x=df['CouncilArea'],y=df['Price'],hue='Type',data=df,palette='Set1')
# g.set_xticklabels(g.get_xticklabels(), rotation=90)
# g.set_title('Preferred Council Area by Price',fontsize=15);
_ = df['CouncilArea']
_ = df['Price']

* Buyers are willing to pay a higher price for houses in council area Boroondara and Stonnington. While lowest in Moorabool.
* Apart from houses being the most sold property,Unit type of property is also being sold in most of the concil areas

In [23]:
# ALEX: remove plotting
# plt.figure(figsize=(10,6))
# g=sns.countplot(df['Regionname'],hue='Type',data=df,palette='Set2')
# g.set_xticklabels(g.get_xticklabels(), rotation=90)
# g.set_title('Preferred region by property type',fontsize=15);

_ = df['Regionname']

* Above plot, we can see preferred regions, categorised by type of the property.There is high sales of all three types of properties in Northern, Western, Southern Metropolitan in comparison to other areas.

In [24]:
# ALEX: remove plotting
# plt.figure(figsize=(10,8))
g=df.groupby(['Regionname'])['Price'].mean().sort_values()
# ALEX: remove plotting
# g=sns.boxplot(x='Regionname', y='Price', data=df,order=list(g.index),palette='cubehelix')
# g=sns.stripplot(x='Regionname', y='Price', data=df,color='orange',jitter=0.2,size=2.5)
# g.set_xticklabels(g.get_xticklabels(), rotation=90)
# plt.title('Price with respect to Region',fontsize=15);
_ = list(g.index)

* We can see that preferred Regions are Northern, Westeern and Southern Metropolitan, ones as we saw earlier were closer to CBD. Further, preferred Council Area are Moreland and Boroondara. 

* If we look by price point then most expensive properties are located in Southern Metropolitan.

* Also, most common types of property in these prefrred regions are houses/villa.

In [25]:
# ALEX: remove plotting
# fig, axs = plt.subplots(ncols=2,figsize=(15,6))

# sns.boxplot(x=df['Method'],y=df['Price'],ax=axs[0],palette='cubehelix').set_title('Price vs Method')

# sns.stripplot(x=df['Type'],y=df['Price'],ax=axs[1],palette='hls').set_title('Price vs Type');
_ = df['Method']
_ = df['Price']
_ = df['Type']
_ = df['Price']

* Houses/ villas are present from low to high price range.

* Not much of a difference can be observed within preferred method when it comes to price of the property.

Below are the scatter plots showing preferred locations by lattitude-longitude values

In [26]:
# ALEX: remove plotting
# fig, axarr = plt.subplots(2, 1, figsize=(12, 16))
# fig.suptitle('Preferred Locations',fontsize=15)

# sns.scatterplot(y=df['Lattitude'],x=df['Longtitude'],hue=df['Regionname'],palette='husl',ax=axarr[0]).set_title('Lattitude-Longitude, Regionname')
# sns.scatterplot(y=df['Lattitude'],x=df['Longtitude'],hue=df['Type'],palette='Set1',ax=axarr[1]).set_title('Lattitude-Longitude,Type')
# plt.show()
_ = df['Lattitude']
_ = df['Longtitude']
_ = df['Regionname']
_ = df['Lattitude']
_ = df['Longtitude']
_ = df['Type']

* Property type that is in demand over all mentioned regions is home/villa type. Even in the areas namely, Northern Victoria and Western Victoria where there is overall less sale of properties, this is the only property type that is being sold.

* Let us now see the trend in pricing with respect to the year in which these properties were built.

In [27]:
# ALEX: make notebook run with input scaling
# df=df.drop(index=9968,axis=1)
df=df.drop(index=9968,axis=1,errors='ignore')

In [28]:
# ALEX: remove plotting
# fig, axarr = plt.subplots(2, 1, figsize=(12, 10))

# fig.suptitle('Trend in Price over the years',fontsize=20)

# sns.lineplot(x='YearBuilt',y='Price',data=df,hue='Type',palette='Set1',ax=axarr[0])

# sns.lineplot(x='YearBuilt',y='Price',data=df,hue='Regionname',palette='Set1',ax=axarr[1]);


* In the first plot, we can observe fluctuations in the prices of houses with respect to the years in which they were constructed. Whereas prices of uniits, duplex and townhouses seems to be irrespective of the years they were built.

* If we see by region,price of houses in Easteren Victoria and Northern Victoria does not seem to be affected by the year in which they were built.

* Buyers in general tend to pay higher price for houses which are built before 1950 as compared to the houses built in recent years, this could indicate preference for a particular design architecture of houses build prior to 1950s. 

In [29]:
def d_date(date):
    return(date[-7:])
df['Date']=df['Date'].apply(d_date)

In [30]:
# ALEX: remove plotting
# plt.figure(figsize=(10,8))
# gg=sns.countplot(df['Date'],palette='hls',hue='Type',data=df)
# gg.set_xticklabels(gg.get_xticklabels(), rotation=70)
# gg.set_title('Month and year of highest sale (2016-2017)',fontsize=15);

_ = df['Date']

* Plot shows the sales count for the year 2016 and 2017 with the data being categorised according to type of houses.

* 2016 saw the lowest sale overall in the month of January and highest sales in November.

* Sales were recoreded to be highest in the month of July 2017.

* In general overall highest selling property type was house/villa.

In [31]:
# ALEX: remove plotting
# plt.figure(figsize=(10,8))
g=df.groupby(['SellerG','Type','Method']).Price.sum().sort_values(ascending=False).head(20)
# ALEX: remove plotting
# g.plot(kind='barh',color='orange',title='Top realtors by Type and Method')
# plt.xlabel('Price');



* Plot shows top realtors categorised by the type and method. Nelson has most number of sales of houses by method S.

* This graph also indicates a trend that houses sold were much higher than any other property type.



Atlast let us take a look at overall correlations within different features of the dataset

In [32]:
# ALEX: remove plotting
# plt.figure(figsize=(16,10))
# plt.title('Correlation heatmap', fontsize=20)
corr_matrix=df.corr()
mask = np.zeros_like(corr_matrix)
mask[np.triu_indices_from(mask)] = True
# ALEX: remove plotting
# sns.heatmap(corr_matrix,annot=True ,cbar = True,cmap="YlGnBu",mask=mask);

As we saw in earlier plots:

* There is a strong negative correlation between price and Distance from CBD.

* Similar is the case with year in which houses were constructed. Old properties tend to be expensive.

# Conclusions
* Sale of all three kinds of properties can be seen in Northern Metropolitan, Western Metropolitan and Southern /metropoiltan as these regions are closer to CBD whereas in other areas sales of units/duplex, townhouse is negligible.

* Properties which were built before 1950's tend to be more expensive.These properties are majorly located in Northern, Southern and Eastern Metropolitan area. Most expensive type of old constructed property includes house/villa type.

* Type of property that is sold out most all across the mentioned areas is house/villa type.

**1)Features of commonly sold properties:**
* 2-3 bedrooms
* 1 bathroom
* Preferred type of property-Houses/Villa
* Preferred regions: Northern, Southern, And Western Metropolitan
* Council areas: Moreland, Boroondara, Glen Era,Darebein, Moone Valley

**2)Features of expensive properties:**
* Property type: House/Villas
* Present in Southern Metropolitan
* Are built in the years,1850-1925
* 5 bedrooms
* More than 5 bathrooms
* Council areas: Boroondara and Stonnington
